In [ ]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?
import sys

from siuba import *
import pandas as pd
import geopandas as gpd
import gcsfs
import datetime as dt
import time
import shapely

from rt_analysis import rt_parser
from rt_analysis import rt_filter_map_plot

import shared_utils
from calitp_data_analysis.tables import tbls

# Migrate existing rt_analysis to use the v2 warehouse 

* mostly just changing queries around

In [ ]:
# date for v2 testing
analysis_date = dt.date(2023, 3, 15)

## Does it work?

In [ ]:
rt_parser.OperatorDayAnalysis?

In [ ]:
analysis_date

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
pbar = tqdm()

In [ ]:
import rt_analysis

In [ ]:
rt_day = rt_parser.OperatorDayAnalysis(4, analysis_date, pbar)

In [ ]:
rt_day = rt_filter_map_plot.RtFilterMapper(rt_day.rt_trips, rt_day.stop_delay_view, rt_day.routelines, pbar)

In [ ]:
rt_day.segment_speed_map()

## Before Merging

* use new folder, restore caching, move to rt_utils
* backwards compatibility -- RtFilterMapper shall accept and use v1 for pre-2023 dates
* message change with Chaithra, pre-stage 2 weeks for SFMTA, AC Transit, SDMTS using v1 around Oct date's she's already using
    * example notebook...
    * note that she's welcome to rerun recent dates with v2
* _some time later_
    * advocate to delete v1 tables
    * remove v1 cached data



In [ ]:
# rt_day.export_views_gcs()